In [15]:
#Load the dataset
import pandas as pd
df = pd.read_csv("Alphabets_data.csv")
df

,letter,xbox,ybox,width,height,onpix,xbar,ybar,x2bar,y2bar,xybar,x2ybar,xy2bar,xedge,xedgey,yedge,yedgex
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,D,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,C,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,T,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,S,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [16]:
from sklearn.preprocessing import StandardScaler
import numpy as np

# Separate features and target variable
X = df.drop('letter', axis=1)
y = df['letter']

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Check for missing values
print("Missing values before handling:")
print(df.isnull().sum())

Missing values before handling:
letter    0
xbox      0
ybox      0
width     0
height    0
onpix     0
xbar      0
ybar      0
x2bar     0
y2bar     0
xybar     0
x2ybar    0
xy2bar    0
xedge     0
xedgey    0
yedge     0
yedgex    0
dtype: int64


In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_scaled.shape[1],)), # Input layer and first hidden layer
    Dense(64, activation='relu'), # Second hidden layer
    Dense(26, activation='softmax') # Output layer (26 classes for letters A-Z)
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Display the model summary
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_26"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_78 (Dense)                │ (None, 128)            │         2,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_79 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_80 (Dense)                │ (None, 26)             │         1,690 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,122 (47.35 KB)

 Trainable params: 12,122 (47.35 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print("Data split into training and testing sets.")
print("Training features shape:", X_train.shape)
print("Testing features shape:", X_test.shape)
print("Training target shape:", y_train.shape)
print("Testing target shape:", y_test.shape)

Data split into training and testing sets.
Training features shape: (16000, 16)
Testing features shape: (4000, 16)
Training target shape: (16000,)
Testing target shape: (4000,)


In [19]:
from sklearn.preprocessing import LabelEncoder

# Encode the target variable into numerical labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Train the model
history = model.fit(X_train, y_train_encoded, epochs=10, batch_size=32, validation_split=0.2)

# Make predictions on the test set
predictions = model.predict(X_test)

# The predictions are probabilities for each class, so we take the class with the highest probability
predicted_classes = np.argmax(predictions, axis=1)

# Decode the predicted numerical labels back to letters
predicted_letters = label_encoder.inverse_transform(predicted_classes)

print("Model training complete.")
print("Predictions made on the test set.")

Epoch 1/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.3848 - loss: 2.2764 - val_accuracy: 0.7378 - val_loss: 0.9295
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7666 - loss: 0.8063 - val_accuracy: 0.8222 - val_loss: 0.6406
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8311 - loss: 0.5777 - val_accuracy: 0.8525 - val_loss: 0.5098
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8703 - loss: 0.4454 - val_accuracy: 0.8769 - val_loss: 0.4220
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8929 - loss: 0.3733 - val_accuracy: 0.8891 - val_loss: 0.3711
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9053 - loss: 0.3100 - val_accuracy: 0.8994 - val_loss: 0.3336
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9182 - loss: 0.2732 - val_accuracy: 0.9078 - val_loss: 0.2948
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9317 - loss: 0.2386 - val_accuracy: 0.

In [20]:
# Define the parameter grid for GridSearchCV
param_grid = {
    'model__neurons': [64, 128],  # Number of neurons in the first hidden layer
    'model__activation': ['relu', 'tanh'], # Activation function for hidden layers
    'model__optimizer': ['adam', 'sgd'] # Optimizer
}

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_model(neurons=128, activation='relu', optimizer='adam'):
    model = Sequential([
        Dense(neurons, activation=activation, input_shape=(X_scaled.shape[1],)),
        Dense(neurons // 2, activation=activation), # Add another layer with half the neurons
        Dense(26, activation='softmax') # Output layer
    ])

    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [24]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

# Create a KerasClassifier instance
keras_classifier = KerasClassifier(model=build_model, verbose=0)

# Instantiate GridSearchCV
grid_search = GridSearchCV(estimator=keras_classifier,
                           param_grid=param_grid,
                           scoring='accuracy',
                           cv=3)

# Execute the grid search
grid_search.fit(X_train, y_train_encoded)

# Best parameters
print("Best hyperparameters found:")
print(grid_search.best_params_)

# Best score
print("Best cross-validated accuracy: {:.4f}".format(grid_search.best_score_))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/lo

Best hyperparameters found:
{'model__activation': 'relu', 'model__neurons': 128, 'model__optimizer': 'adam'}
Best cross-validated accuracy: 0.7234


In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Evaluate the model on the test set
accuracy = accuracy_score(y_test_encoded, predicted_classes)
precision = precision_score(y_test_encoded, predicted_classes, average='weighted')
recall = recall_score(y_test_encoded, predicted_classes, average='weighted')
f1 = f1_score(y_test_encoded, predicted_classes, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Accuracy: 0.9293
Precision: 0.9321
Recall: 0.9293
F1-score: 0.9296
